In [272]:
import pandas as pd
from pathlib import Path
import re
import nltk
import numpy as np
from collections import defaultdict
import ast

## Annonymize UPMC

In [265]:
data_dir = Path("/ocean/projects/asc170022p/shg121/PhD/Mammo-CLIP/src/codebase/data_csv")
csv = "upmc_dicom_consolidated_final_folds_BIRADS_num_1_report.csv"
df = pd.read_csv(data_dir/csv)
print(df.shape)
df.head(4)

(76251, 21)


,patient_id,GENDER,AGE,NOTE_CSN_STUDY_ID,BIRADS_CATEGORY,HISTORY,FINDINGS,IMPRESSION,ASSESMENT,RECOMENDATION,...,CLIP,image_id,fold,view,laterality,img_size_row,img_size_column,BIRADS_numeric,REPORT,text
0,40421851,FEMALE,47,884708601,BIRADS 0,NaN,There is a small oval asymmetry anterior de...,BI-RADS CATEGORY 0. The patient is 47 years ol...,NaN,NaN,...,0,1.2.840.113654.2.70.1.250833044448633383274960...,2,CC,R,2456,1995,0,There is a small oval asymmetry anterior de...,there is a small oval asymmetry anterior depth...
1,40421851,FEMALE,47,884708601,BIRADS 0,NaN,There is a small oval asymmetry anterior de...,BI-RADS CATEGORY 0. The patient is 47 years ol...,NaN,NaN,...,0,1.2.840.113654.2.70.1.218018924783571581641971...,2,CC,R,2456,1995,0,There is a small oval asymmetry anterior de...,there is a small oval asymmetry anterior depth...
2,936277906,FEMALE,78,831725898,BIRADS 2,The patient is 78 years old and presents fo...,Bilateral digital mammography with CAD\nrev...,BI-RADS CATEGORY 2. The patient is 78 years ol...,BI-RADS: 2 - BENIGN FINDING(S)\n\nRIS CODE: ...,\nRoutine screening mammography or combinatio...,...,0,1.2.840.113654.2.70.1.159160858633911675764962...,2,CC,R,4096,3328,2,Bilateral digital mammography with CAD\nrev...,bilateral digital mammography with cad reveals...
3,936277906,FEMALE,78,831725898,BIRADS 2,The patient is 78 years old and presents fo...,Bilateral digital mammography with CAD\nrev...,BI-RADS CATEGORY 2. The patient is 78 years ol...,BI-RADS: 2 - BENIGN FINDING(S)\n\nRIS CODE: ...,\nRoutine screening mammography or combinatio...,...,0,1.2.840.113654.2.70.1.104389345981935981355845...,2,MLO,R,4096,3328,2,Bilateral digital mammography with CAD\nrev...,bilateral digital mammography with cad reveals...


In [267]:
import pandas as pd
import random

df = df.head(100)
# Function to generate synthetic report text
def generate_synthetic_text(template):
    conditions = [
        "no significant abnormality detected",
        "benign calcifications present",
        "suspicious mass identified requiring further evaluation",
        "asymmetrical breast density observed",
        "fibroglandular densities noted",
        "microcalcifications detected in the upper quadrant",
        "architectural distortion with no associated mass",
        "lymph node with cortical thickening",
        "solitary dilated duct visible",
        "biopsy recommended for suspicious lesion",
        "implants intact with no evidence of rupture",
        "post-surgical changes evident",
        "benign lymph node enlargement",
        "diffuse skin thickening",
        "possible intramammary lymph node",
        "fibrocystic changes with multiple cysts",
        "normal axillary appearance",
        "previous excisional biopsy site noted",
        "scattered fibroglandular tissue"
    ]
    findings = random.choice(conditions)
    return template.replace("{findings}", findings)

# Function to anonymize findings and impressions
def anonymize_reports(df, id_cols, text_cols):
    anonymized_dict = {}  # Dictionary to store original and anonymized mappings
    for index, row in df.iterrows():
        key = tuple(row[col] for col in id_cols)  # Create a unique key for each patient_id and laterality
        for col in text_cols:
            if key + (row[col],) not in anonymized_dict:
                # Generate synthetic text based on a template
                template = "The examination shows {findings}. Further examination may be needed."
                synthetic_text = generate_synthetic_text(template)
                anonymized_dict[key + (row[col],)] = synthetic_text
            # Replace the original text with the synthetic text
            df.at[index, col] = anonymized_dict[key + (row[col],)]

    return df

# Anonymize the data
id_columns = ['patient_id', 'laterality']
text_columns = ["FINDINGS"]
df = anonymize_reports(df, id_columns, text_columns)

# df.drop('REPORT', axis=1, inplace=True)
# df.drop('text', axis=1, inplace=True)

In [268]:
import pandas as pd
import random

# Possible conditions for the random text in impressions
impression_texts = [
    "further imaging evaluation is suggested",
    "routine mammographic screening recommended",
    "patient should consider clinical follow-up",
    "follow-up and possible biopsy necessary",
    "no additional imaging required at this time",
    "recommend MRI for better evaluation"
]

# Function to generate synthetic impressions
def generate_synthetic_impressions():
    birads = random.randint(0, 6)  # Random BIRADS number between 0 and 6
    age = random.randint(25, 90)  # Random age between 25 and 90
    text = random.choice(impression_texts)  # Random text from predefined phrases
    return f"Birads: {birads}, Patient is: {age} years old, and {text}"

# Function to anonymize reports specifically for the impressions column
def anonymize_impressions(df, id_cols, text_col):
    anonymized_dict = {}  # Dictionary to store mappings
    for index, row in df.iterrows():
        key = tuple(row[col] for col in id_cols)  # Create a unique key based on patient_id and laterality
        if key not in anonymized_dict:
            # Generate synthetic impressions text
            synthetic_text = generate_synthetic_impressions()
            anonymized_dict[key] = synthetic_text
        # Replace the original text with the synthetic text
        df.at[index, text_col] = anonymized_dict[key]

    return df

# Example DataFrame loading and processing
id_columns = ['patient_id', 'laterality']
df = anonymize_impressions(df, id_columns, 'IMPRESSION')

In [269]:
df

,patient_id,GENDER,AGE,NOTE_CSN_STUDY_ID,BIRADS_CATEGORY,HISTORY,FINDINGS,IMPRESSION,ASSESMENT,RECOMENDATION,CONCLUSION,CLIP,image_id,fold,view,laterality,img_size_row,img_size_column,BIRADS_numeric
0,40421851,FEMALE,47,884708601,BIRADS 0,NaN,The examination shows biopsy recommended for s...,"Birads: 4, Patient is: 47 years old, and furth...",NaN,NaN,NaN,0,1.2.840.113654.2.70.1.250833044448633383274960...,2,CC,R,2456,1995,0
1,40421851,FEMALE,47,884708601,BIRADS 0,NaN,The examination shows biopsy recommended for s...,"Birads: 4, Patient is: 47 years old, and furth...",NaN,NaN,NaN,0,1.2.840.113654.2.70.1.218018924783571581641971...,2,CC,R,2456,1995,0
2,936277906,FEMALE,78,831725898,BIRADS 2,The patient is 78 years old and presents fo...,The examination shows no significant abnormali...,"Birads: 1, Patient is: 48 years old, and routi...",BI-RADS: 2 - BENIGN FINDING(S)\n\nRIS CODE: ...,\nRoutine screening mammography or combinatio...,NaN,0,1.2.840.113654.2.70.1.159160858633911675764962...,2,CC,R,4096,3328,2
3,936277906,FEMALE,78,831725898,BIRADS 2,The patient is 78 years old and presents fo...,The examination shows no significant abnormali...,"Birads: 1, Patient is: 48 years old, and routi...",BI-RADS: 2 - BENIGN FINDING(S)\n\nRIS CODE: ...,\nRoutine screening mammography or combinatio...,NaN,0,1.2.840.113654.2.70.1.104389345981935981355845...,2,MLO,R,4096,3328,2
4,936277906,FEMALE,78,831725898,BIRADS 2,The patient is 78 years old and presents fo...,The examination shows benign calcifications pr...,"Birads: 1, Patient is: 52 years old, and follo...",BI-RADS: 2 - BENIGN FINDING(S)\n\nRIS CODE: ...,\nRoutine screening mammography or combinatio...,NaN,0,1.2.840.113654.2.70.1.239577423760362698062104...,2,CC,L,4096,3328,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,40487189,FEMALE,89,825133346,BIRADS 1,NaN,The examination shows diffuse skin thickening....,"Birads: 5, Patient is: 86 years old, and patie...",BI-RADS: 1 - NEGATIVE\n\nRIS CODE: 2NEGATIVE\n,NaN,NaN,0,1.2.840.113654.2.70.1.225416017428424459803050...,2,CC,L,2457,1996,1
96,40487189,FEMALE,89,825133346,BIRADS 1,NaN,The examination shows architectural distortion...,"Birads: 5, Patient is: 28 years old, and patie...",BI-RADS: 1 - NEGATIVE\n\nRIS CODE: 2NEGATIVE\n,NaN,NaN,0,1.2.840.113654.2.70.1.175010212751333265973867...,2,CC,R,2457,1996,1
97,40487189,FEMALE,89,825133346,BIRADS 1,NaN,The examination shows architectural distortion...,"Birads: 5, Patient is: 28 years old, and patie...",BI-RADS: 1 - NEGATIVE\n\nRIS CODE: 2NEGATIVE\n,NaN,NaN,0,1.2.840.113654.2.70.1.199035556945135657403747...,2,MLO,R,2457,1996,1
98,40487189,FEMALE,89,825133346,BIRADS 1,NaN,The examination shows diffuse skin thickening....,"Birads: 5, Patient is: 86 years old, and patie...",BI-RADS: 1 - NEGATIVE\n\nRIS CODE: 2NEGATIVE\n,NaN,NaN,0,1.2.840.113654.2.70.1.225334922081844040972632...,2,MLO,L,2457,1996,1


In [270]:
df.to_csv(
    data_dir/csv, 
    index=False
)

## Read final data

In [303]:
new_df = pd.read_csv(data_dir/"upmc_breast_clip_without_period_lower_case.csv")
print(new_df.shape)
new_df["text"][0], new_df["text_augment"][0]


(42, 11)


("['the examination shows benign lymph node enlargement further examination may be needed', 'birads : 4 , patient is : 69 years old , and no additional imaging required at this time']",
 "['the examination shows a benign enlargement of the lymph node may require a further examination', 'birads: 4, patient is: 69 years old, and no further imaging required at this time']")

In [304]:
new_df.columns

Index(['patient_id', 'laterality', 'image', 'view', 'CC', 'MLO', 'text',
       'findings', 'impressions', 'text_augment', 'fold'],
      dtype='object')

In [305]:
new_df

,patient_id,laterality,image,view,CC,MLO,text,findings,impressions,text_augment,fold
0,40408552,R,['1.2.840.113654.2.70.1.1407279823421617265390...,"['CC', 'MLO']",['1.2.840.113654.2.70.1.8677170909164737970683...,['1.2.840.113654.2.70.1.1407279823421617265390...,['the examination shows benign lymph node enla...,the examination shows benign lymph node enlarg...,"birads : 4 , patient is : 69 years old , and n...",['the examination shows a benign enlargement o...,1
1,40409490,L,['1.2.840.113654.2.70.1.9471108146030766369773...,"['CC', 'MLO']",['1.2.840.113654.2.70.1.9471108146030766369773...,['1.2.840.113654.2.70.1.2120553192244630999778...,['the examination shows benign calcifications ...,the examination shows benign calcifications pr...,"birads : 6 , patient is : 47 years old , and p...",['the examination results in benign calcinatio...,2
2,40409490,R,['1.2.840.113654.2.70.1.2742796411782465870559...,"['CC', 'MLO']",['1.2.840.113654.2.70.1.2146099278055958953117...,['1.2.840.113654.2.70.1.2742796411782465870559...,['the examination shows implants intact with n...,the examination shows implants intact with no ...,"birads : 3 , patient is : 72 years old , and r...",['the examination shows intact implants withou...,2
3,40421851,R,['1.2.840.113654.2.70.1.2508330444486333832749...,['CC'],['1.2.840.113654.2.70.1.2508330444486333832749...,[],['the examination shows biopsy recommended for...,the examination shows biopsy recommended for s...,"birads : 4 , patient is : 47 years old , and f...",['examination shows recommended biopsy for sus...,0
4,40487189,L,['1.2.840.113654.2.70.1.2254160174284244598030...,"['CC', 'MLO']",['1.2.840.113654.2.70.1.2254160174284244598030...,['1.2.840.113654.2.70.1.2253349220818440409726...,['the examination shows diffuse skin thickenin...,the examination shows diffuse skin thickening ...,"birads : 5 , patient is : 86 years old , and p...",['examination shows widespread skin thickening...,3
5,40487189,R,['1.2.840.113654.2.70.1.1750102127513332659738...,"['CC', 'MLO']",['1.2.840.113654.2.70.1.1750102127513332659738...,['1.2.840.113654.2.70.1.1990355569451356574037...,['the examination shows architectural distorti...,the examination shows architectural distortion...,"birads : 5 , patient is : 28 years old , and p...",['exam demonstrates architectural distortion w...,3
6,40494117,L,['1.2.840.113654.2.70.1.7853263274101029885375...,"['CC', 'MLO']",['1.2.840.113654.2.70.1.2278802375545091134820...,['1.2.840.113654.2.70.1.7853263274101029885375...,['the examination shows scattered fibroglandul...,the examination shows scattered fibroglandular...,"birads : 6 , patient is : 77 years old , and n...",['examination shows fibroglandular tissue spre...,0
7,40494117,R,['1.2.840.113654.2.70.1.2624516273792895953248...,"['CC', 'MLO']",['1.2.840.113654.2.70.1.1711132321043880122014...,['1.2.840.113654.2.70.1.2624516273792895953248...,['the examination shows diffuse skin thickenin...,the examination shows diffuse skin thickening ...,"birads : 6 , patient is : 56 years old , and r...",['the examination shows a widespread skin thic...,0
8,40496778,L,['1.2.840.113654.2.70.1.4497059171488859636768...,"['CC', 'MLO']",['1.2.840.113654.2.70.1.2726132503524300162928...,['1.2.840.113654.2.70.1.4497059171488859636768...,['the examination shows no significant abnorma...,the examination shows no significant abnormali...,"birads : 1 , patient is : 85 years old , and r...",['the examination shows no significant abnorma...,1
9,40496778,R,['1.2.840.113654.2.70.1.1556578082456049283440...,"['CC', 'MLO']",['1.2.840.113654.2.70.1.1922104217140782300547...,['1.2.840.113654.2.70.1.1556578082456049283440...,['the examination shows fibrocystic changes wi...,the examination shows fibrocystic changes with...,"birads : 0 , patient is : 61 years old , and f...",['examination shows fibrocystic changes with m...,1


In [306]:
# Print the DataFrame to check 'fold' assignment
print()

# Check that each patient_id is only in one fold
if new_df.groupby('patient_id')['fold'].nunique().max() == 1:
    print("All patient_id's are correctly assigned to a single fold.")
else:
    print("Some patient_id's are split across multiple folds.")



All patient_id's are correctly assigned to a single fold.
